In [13]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

# citeseer_dataset = Planetoid(root = "./data", name = "CiteSeer")
cora_dataset = Planetoid(root = "./data", name = "Cora")
# pubmed_dataset = Planetoid(root = "./data", name = "PubMed")

dataset = cora_dataset

In [14]:
dataset

Cora()

In [15]:
import argparse

import torch
import torch.nn.functional as F
from citation import get_planetoid_dataset, random_planetoid_splits, run

from torch_geometric.nn import GCNConv
from torch_geometric.profile import rename_profile_file


class Net(torch.nn.Module):
    def __init__(self, dataset, hidden_dim, dropout):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, dataset.num_classes)
        self.dropout = dropout

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [76]:
device= torch.device("cuda:0")

In [77]:
model = Net(dataset, 64, 0.8).to(device)
optim = torch.optim.Adam(params=model.parameters(), lr = 1e-2, weight_decay=1e-3)
loss_fun = torch.nn.CrossEntropyLoss()

In [79]:
from sklearn.model_selection import KFold
import numpy as np
from tqdm.notebook import tqdm

k_outer = 100
k_inner = 5
kf_outer = KFold(n_splits=k_outer)
EARLY_STOPPING = 50

scores = torch.zeros(k_outer)

def index_to_mask(rows, index_array):
    mask_array = np.zeros(rows, dtype=int)
    mask_array[index_array] = 1
    return torch.from_numpy(mask_array.astype(np.bool_))

def evaluate(model, test_mask):
    global dataset
    with torch.inference_mode():
        model.eval()
        out = model(dataset[0].to(device))
        loss = loss_fun(out[test_mask], dataset[0].y[test_mask].to(device))
        acc = torch.sum(out[test_mask].argmax(1) == dataset[0].y[test_mask].to(device)) / test_mask.sum()
        return acc, loss.item()
        
    
for i, (train_index, test_index) in tqdm(enumerate(kf_outer.split(dataset[0].x.numpy()))):            
    outer_train_mask = index_to_mask(dataset[0].x.shape[0], train_index)
    outer_test_mask = index_to_mask(dataset[0].x.shape[0], test_index)
    kf_inner = KFold(n_splits=k_inner)
    best_val_scores = torch.zeros(k_inner)
    has_breaked = False
    for j, (train_index_inner, test_index_inner) in tqdm(enumerate(kf_inner.split(dataset[0].x.numpy()[train_index]))): ##self.outer_train_index
        
        inner_train_mask = index_to_mask(dataset[0].x.shape[0], train_index[train_index_inner])
        sample_mask = get_sample_mask(inner_train_mask)
        
        inner_test_mask = index_to_mask(dataset[0].x.shape[0], train_index[test_index_inner])
        val_losses = []
        val_accuracies = []
        for epoch in tqdm(range(10_000)): ##TODO: 100_000
            model.train()
            out = model(dataset[0].to(device))
            loss = loss_fun(out[inner_train_mask][sample_mask], dataset[0].y[inner_train_mask][sample_mask].to(device))
            optim.zero_grad()
            loss.backward()
            optim.step()
            acc_val, val_loss = evaluate(model, inner_test_mask)
            val_losses.append(val_loss)
            val_accuracies.append(acc_val)
            if len(val_losses) > EARLY_STOPPING and all(list(map(lambda prev_val_loss: prev_val_loss <= val_loss, val_losses))):
                has_breaked = True
                break
        best_val_scores[j] = val_accuracies[-EARLY_STOPPING] if has_breaked else val_accuracies[-1]
    print(best_val_scores)
    scores[i], _ = evaluate(model, outer_test_mask)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

tensor([0.7821, 0.7649, 0.8078, 0.8097, 0.8078])


In [80]:
scores.mean(), scores.std()

(tensor(0.7816), tensor(0.0859))

In [73]:
def get_sample_mask(inner_mask, sample_size_per_class = 20):
    global dataset
    class_train_masks = []
    for class_i in range(dataset.num_classes):
        class_train_mask = (dataset[0].y[inner_mask] == class_i)
        class_train_indices = class_train_mask.nonzero()
        indices = torch.randperm(class_train_mask.sum())[:sample_size_per_class]
        class_train_indices = class_train_indices[indices]
        class_train_mask = torch.zeros_like(class_train_mask)
        class_train_mask[class_train_indices] = 1
        class_train_masks.append(class_train_mask)
        # print(dataset[0].y[inner_mask][class_train_mask])
    sample_mask = torch.stack(class_train_masks).sum(0).type(torch.bool)    
    return sample_mask

In [68]:
np.unique(dataset[0].y[outer_train_mask][sample_mask].numpy(), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]), array([20, 20, 20, 20, 20, 20, 20]))

In [65]:
torch.stack(class_train_masks).nonzero()

tensor([[   0,   70],
        [   0,  412],
        [   0,  578],
        [   0,  638],
        [   0,  936],
        [   0,  997],
        [   0, 1014],
        [   0, 1051],
        [   0, 1111],
        [   0, 1251],
        [   0, 1877],
        [   0, 1991],
        [   0, 2105],
        [   0, 2109],
        [   0, 2158],
        [   0, 2191],
        [   0, 2263],
        [   0, 2283],
        [   0, 2474],
        [   0, 2483],
        [   1,    8],
        [   1,  208],
        [   1,  275],
        [   1,  378],
        [   1,  459],
        [   1,  803],
        [   1,  917],
        [   1,  930],
        [   1, 1114],
        [   1, 1255],
        [   1, 1276],
        [   1, 1477],
        [   1, 1556],
        [   1, 1642],
        [   1, 1806],
        [   1, 2058],
        [   1, 2059],
        [   1, 2180],
        [   1, 2349],
        [   1, 2563],
        [   2,   44],
        [   2,  262],
        [   2,  331],
        [   2,  336],
        [   2,  734],
        [ 